# Notebook 3: Combining Fangraphs and Baseball Reference Data

### Introduction

From the previous two notebooks, we now have clean projection and season data to use for modeling. In this notebook, I'll combine the relevant datasets to form the training, validation, and test set.

In [1]:
import pandas as pd

### Data for Training Model

My training set will use season data from 2016 and projection data from 2017 to predict quality starts in 2017. This code goes over how I combine them all, and what rows/players were dropped.

 Need to have:
- season QS data for 2017
- projection (all) data for 2017
- season (all) data for 2016

In [2]:
qs_17 = pd.read_csv('../data/season_2017_c.csv')
qs_17_simple = qs_17[['Full_Name', 'ID', 'QS_2017']]

In [3]:
season_2016 = pd.read_csv('../data/season_2016_c.csv')

This dataset (s16_qs17) has a QS_2017 value for all rows. However, not all rows will have 2016 season data. This is all taken from baseball reference.

In [4]:
s16_qs17 = season_2016.merge(qs_17_simple, left_on =['ID', 'Full_Name'], 
                                    right_on = ['ID', 'Full_Name'], how = 'right')
# Performing a right join here means that all rows have a QS_2017 value, and not all rows have 2016 season data.

In [5]:
s16_qs17.shape

(316, 38)

Now we will read in the projection data and perform the first match, on player ID. Note that player ID has been constructed in the Fangraphs projection data based on the Player Name. It will not necessarily match the ID given by Baseball Reference, and will need to be updated for non-matches.

In [6]:
proj17 = pd.read_csv('../data/projected_2017_c.csv')

In [7]:
train_matchID = s16_qs17.merge(proj17, left_on =['ID'], 
                                    right_on = ['ID'], how = 'inner')


This dataset is the result of the first match between the 2017QS/2016 season data and the 2017 projection data.

In [8]:
train_matchID

,Full_Name,ID,Age_2016,Tm_2016,IP_2016,G_2016,GS_2016,Wgs_2016,Lgs_2016,ND_2016,...,BABIP_2017,ERA_2017,FIP_2017,ERA-_2017,FIP-_2017,K/9_2017,BB/9_2017,HR/9_2017,ERA+_2017,zWAR_2017
0,Tim Adleman,adlemti01,28.0,CIN,69.2,13.0,13.0,4.0,4.0,5.0,...,0.285,4.76,4.97,113,116,6.61,2.98,1.45,84,0.4
1,Raul Alcantara,alcanra01,23.0,OAK,22.1,5.0,5.0,1.0,3.0,1.0,...,0.296,5.10,5.00,126,121,5.42,2.28,1.49,78,0.0
2,Chase Anderson,anderch01,28.0,MIL,151.2,31.0,30.0,9.0,11.0,10.0,...,0.293,4.64,4.69,108,108,7.28,2.96,1.42,88,1.1
3,Tyler Anderson,anderty01,26.0,COL,114.1,19.0,19.0,5.0,6.0,8.0,...,0.302,4.28,4.47,87,99,7.05,3.09,1.19,110,2.3
4,Matt Andriese,andrima01,26.0,TBR,127.2,29.0,19.0,7.0,7.0,5.0,...,0.293,3.71,3.49,91,86,7.93,1.89,1.01,106,2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Adam Wilk,wilkad01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.296,4.78,4.62,117,113,6.29,2.47,1.35,83,0.3
260,Adam Wilk,wilkad01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.291,5.10,4.93,129,118,6.29,2.47,1.51,74,-0.5
261,Aaron Wilkerson,wilkeaa01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.297,4.43,4.32,103,100,8.52,3.18,1.25,92,1.3
262,Travis Wood,woodtr01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.271,3.39,3.91,82,94,8.56,3.54,1.03,116,0.5


Let's capture the rest of the unmatched data, where there was no ID that lined up.

In [9]:
train_matchID_unmatched_bbref = s16_qs17[~s16_qs17.ID.isin(train_matchID.ID)]
train_matchID_unmatched_proj  = proj17[~proj17.ID.isin(train_matchID.ID)].drop(['ID'], axis = 1)

Now let's perform the second match, on Full_Name. 

In [10]:
train_match_Full_Name = train_matchID_unmatched_bbref.merge(train_matchID_unmatched_proj, left_on =['Full_Name'], 
                                    right_on = ['Player'], how = 'inner')

In [11]:
train_match_Full_Name

,Full_Name,ID,Age_2016,Tm_2016,IP_2016,G_2016,GS_2016,Wgs_2016,Lgs_2016,ND_2016,...,BABIP_2017,ERA_2017,FIP_2017,ERA-_2017,FIP-_2017,K/9_2017,BB/9_2017,HR/9_2017,ERA+_2017,zWAR_2017
0,Brett Anderson,anderbr04,28.0,LAD,11.1,4.0,3.0,0.0,2.0,1.0,...,0.291,4.44,4.29,112,105,6.04,2.39,1.14,85,0.4
1,Homer Bailey,baileho02,30.0,CIN,23.0,6.0,6.0,2.0,3.0,1.0,...,0.297,4.50,4.28,107,100,7.67,2.76,1.23,89,0.6
2,Wei-Yin Chen,chenwe02,30.0,MIA,123.1,22.0,22.0,5.0,5.0,12.0,...,0.283,3.73,3.97,91,98,7.16,1.90,1.16,106,2.4
3,Zach Davies,davieza02,23.0,MIL,163.1,28.0,28.0,11.0,7.0,10.0,...,0.292,3.99,3.95,93,91,7.76,2.55,1.06,102,2.6
4,R.A. Dickey,dicker.01,41.0,TOR,169.2,30.0,29.0,10.0,14.0,5.0,...,0.273,4.50,4.71,109,116,6.19,3.21,1.17,88,1.1
5,Jaime Garcia,garcija02,29.0,STL,171.2,32.0,30.0,10.0,13.0,7.0,...,0.292,4.03,4.11,98,102,7.99,2.62,1.20,98,1.6
6,Jaime Garcia,garcija02,29.0,STL,171.2,32.0,30.0,10.0,13.0,7.0,...,0.292,4.03,4.11,98,102,7.99,2.62,1.20,98,1.6
7,Miguel Gonzalez,gonzami03,32.0,CHW,135.0,24.0,23.0,5.0,8.0,10.0,...,0.297,4.63,4.63,109,107,6.95,2.78,1.39,89,1.2
8,Jon Gray,grayjo02,24.0,COL,168.0,29.0,29.0,10.0,10.0,9.0,...,0.309,4.18,4.06,85,90,8.83,3.08,1.16,112,3.2
9,Chad Green,greench03,25.0,NYY,45.2,12.0,8.0,2.0,4.0,2.0,...,0.303,4.69,4.47,110,102,8.25,2.87,1.47,87,0.8


In [12]:
train_matchID_unmatched_bbref_r2 = s16_qs17[~s16_qs17.ID.isin(train_matchID.ID) & ~s16_qs17.ID.isin(train_match_Full_Name.ID)]
train_matchID_unmatched_proj_r2  = proj17[~proj17.ID.isin(train_matchID.ID) & ~proj17.ID.isin(train_match_Full_Name.ID)]

In [13]:
#Can confirm, none of these players appeared in the 2017 projection data.
train_matchID_unmatched_bbref_r2

,Full_Name,ID,Age_2016,Tm_2016,IP_2016,G_2016,GS_2016,Wgs_2016,Lgs_2016,ND_2016,...,RS/GS_2016,RS/IP_2016,IP/GS_2016,Pit/GS_2016,<80_2016,80-99_2016,100-119_2016,≥120_2016,Max_2016,QS_2017
1,Andrew Albers,alberan01,30.0,MIN,17.0,6.0,2.0,0.0,0.0,2.0,...,6.4,14.1,3.3,73.0,1.0,1.0,0.0,0.0,88.0,2
29,Jhoulys Chacin,chacijh01,28.0,TOT,144.0,34.0,22.0,5.0,8.0,9.0,...,4.6,3.5,5.2,87.0,4.0,17.0,1.0,0.0,114.0,16
38,Patrick Corbin,corbipa01,26.0,ARI,155.2,36.0,24.0,4.0,12.0,8.0,...,4.7,4.4,5.5,91.0,3.0,17.0,4.0,0.0,108.0,19
57,Yovani Gallardo,gallayo01,30.0,BAL,118.0,23.0,23.0,6.0,8.0,9.0,...,4.6,4.5,5.1,92.0,2.0,15.0,6.0,0.0,108.0,5
117,Lance McCullers Jr.,mcculla02,22.0,HOU,81.0,14.0,14.0,6.0,5.0,3.0,...,3.2,3.3,5.8,96.0,1.0,9.0,4.0,0.0,117.0,9
183,Dan Straily,straida01,27.0,CIN,191.1,34.0,31.0,14.0,8.0,9.0,...,5.3,4.3,5.9,94.0,4.0,18.0,9.0,0.0,112.0,12
197,Jacob Turner,turneja01,25.0,CHW,24.2,18.0,2.0,0.0,1.0,1.0,...,3.0,2.3,3.7,91.0,0.0,1.0,1.0,0.0,101.0,1
219,Chris Young,youngch03,37.0,KCR,88.2,34.0,13.0,1.0,8.0,4.0,...,2.9,2.5,4.3,82.0,4.0,8.0,1.0,0.0,101.0,0
223,Bronson Arroyo,arroybr01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
242,Dayan Diaz,diazda01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


Now that I've found all possible player matches, I'll finalize the training data, and output it as a csv.

In [14]:
training_data_model = pd.concat([train_matchID, train_match_Full_Name], ignore_index = True, axis = 0)
#training_data_model.to_csv('training_data_model.csv', index = False)

### Data for Validation Model

My validation set will use season data from 2017 and projection data from 2018 to predict quality starts in 2018. This code goes over how I combine them all, and what rows/players were dropped. This is similar to what I did for the training data, but for a different year.

 Need to have:
- season QS data for 2018
- projection (all) data for 2018
- season (all) data for 2017

In [15]:
qs_18 = pd.read_csv('../data/season_2018_c.csv')
qs_18_simple = qs_18[['Full_Name', 'ID', 'QS_2018']]

In [16]:
season_2017 = pd.read_csv('../data/season_2017_c.csv')
s17_qs18 = season_2017.merge(qs_18_simple, left_on =['ID', 'Full_Name'], 
                                    right_on = ['ID', 'Full_Name'], how = 'right')

In [17]:
proj18 = pd.read_csv('../data/projected_2018_c.csv')

In [18]:
train_matchID = s17_qs18.merge(proj18, left_on =['ID'], 
                                    right_on = ['ID'], how = 'inner')

In [19]:
train_matchID_unmatched_bbref = s17_qs18[~s17_qs18.ID.isin(train_matchID.ID)]
train_matchID_unmatched_proj  = proj18[~proj18.ID.isin(train_matchID.ID)].drop(['ID'], axis = 1)

In [20]:
train_match_Full_Name = train_matchID_unmatched_bbref.merge(train_matchID_unmatched_proj, left_on =['Full_Name'], 
                                    right_on = ['Player'], how = 'inner')

In [21]:
train_matchID_unmatched_bbref_r2 = s17_qs18[~s17_qs18.ID.isin(train_matchID.ID) & ~s17_qs18.ID.isin(train_match_Full_Name.ID)]
train_matchID_unmatched_proj_r2  = proj18[~proj18.ID.isin(train_matchID.ID) & ~proj18.ID.isin(train_match_Full_Name.ID)]

In [22]:
train_matchID_unmatched_bbref_r2

,Full_Name,ID,Age_2017,Tm_2017,IP_2017,G_2017,GS_2017,Wgs_2017,Lgs_2017,ND_2017,...,RS/GS_2017,RS/IP_2017,IP/GS_2017,Pit/GS_2017,<80_2017,80-99_2017,100-119_2017,≥120_2017,Max_2017,QS_2018
119,Lance McCullers Jr.,mcculla02,23.0,HOU,118.2,22.0,22.0,7.0,4.0,11.0,...,4.9,3.7,5.4,92.0,3.0,15.0,4.0,0.0,105.0,13
130,Matt Moore,moorema02,28.0,SFG,174.1,32.0,31.0,6.0,15.0,10.0,...,3.7,3.3,5.6,92.0,5.0,18.0,7.0,1.0,120.0,2
132,Joe Musgrove,musgrjo01,24.0,HOU,109.1,38.0,15.0,4.0,8.0,3.0,...,4.2,4.0,5.2,88.0,2.0,11.0,2.0,0.0,102.0,10
196,Jacob Turner,turneja01,26.0,WSN,39.0,18.0,2.0,0.0,1.0,1.0,...,2.5,3.0,5.7,87.0,0.0,2.0,0.0,0.0,89.0,0
218,Scott Alexander,alexasc02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
229,Jeff Brigham,brighje01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
238,Enyel De Los Santos,delosen01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
245,Caleb Ferguson,ferguca01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
246,Matt Festa,festama01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
251,Drew Gagnon,gagnodr01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [23]:
validation_data_model = pd.concat([train_matchID, train_match_Full_Name], ignore_index = True, axis = 0)
#validation_data_model.to_csv('validation_data_model.csv', index = False)

### Data for Test Model

My test set will use season data from 2018 and projection data from 2019 to predict quality starts in 2019. This code goes over how I combine them all, and what rows/players were dropped. This is similar to what I did for the training and validation data, but for a different year.
Need to have:

- season QS data for 2019
- projection (all) data for 2019
- season (all) data for 2018

In [24]:
qs_19 = pd.read_csv('../data/season_2019_c.csv')
qs_19_simple = qs_19[['Full_Name', 'ID', 'QS_2019']]

In [25]:
season_2018 = pd.read_csv('../data/season_2018_c.csv')


In [26]:
s18_qs19 = season_2018.merge(qs_19_simple, left_on =['ID', 'Full_Name'], 
                                    right_on = ['ID', 'Full_Name'], how = 'right')

In [27]:
proj19 = pd.read_csv('../data/projected_2019_c.csv')

In [28]:
train_matchID = s18_qs19.merge(proj19, left_on =['ID'], 
                                    right_on = ['ID'], how = 'inner')

In [29]:
train_matchID_unmatched_bbref = s18_qs19[~s18_qs19.ID.isin(train_matchID.ID)]
train_matchID_unmatched_proj  = proj19[~proj19.ID.isin(train_matchID.ID)].drop(['ID'], axis = 1)

In [30]:
train_match_Full_Name = train_matchID_unmatched_bbref.merge(train_matchID_unmatched_proj, left_on =['Full_Name'], 
                                    right_on = ['Player'], how = 'inner')

In [31]:
train_matchID_unmatched_bbref_r2 = s18_qs19[~s18_qs19.ID.isin(train_matchID.ID) & ~s18_qs19.ID.isin(train_match_Full_Name.ID)]
train_matchID_unmatched_proj_r2  = proj19[~proj19.ID.isin(train_matchID.ID) & ~proj19.ID.isin(train_match_Full_Name.ID)]

In [32]:
train_matchID_unmatched_bbref_r2

,Full_Name,ID,Age_2018,Tm_2018,IP_2018,G_2018,GS_2018,Wgs_2018,Lgs_2018,ND_2018,...,RS/GS_2018,RS/IP_2018,IP/GS_2018,Pit/GS_2018,<80_2018,80-99_2018,100-119_2018,≥120_2018,Max_2018,QS_2019
7,Homer Bailey,baileho02,32.0,CIN,106.1,20.0,20.0,1.0,14.0,5.0,...,2.8,1.6,5.3,89.0,2.0,14.0,4.0,0.0,115.0,13
99,Brian Johnson,johnsbr02,27.0,BOS,99.1,38.0,13.0,4.0,3.0,6.0,...,6.8,7.3,4.7,78.0,6.0,6.0,1.0,0.0,104.0,0
233,Austin Adams,adamsau02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
252,Ronald Bolanos,bolanro01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
261,Matt Carasiti,carasma01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
266,Emmanuel Clase,claseem01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
270,Randy Dobnak,dobnara01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
275,Montana DuRapau,durapmo01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
276,Tom Eshelman,eshelto01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
277,Ryan Feierabend,feierry01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [33]:
test_data_model = pd.concat([train_matchID, train_match_Full_Name], ignore_index = True, axis = 0)
#test_data_model.to_csv('test_data_model.csv', index = False)

In [34]:
test_data_model.shape

(359, 62)